In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
pip install jinja2

In [0]:
from jinja2 import Template

In [0]:
parameter = [
    {
        "table" : "spotifycatalog.silver.factstream",
        "alias" : "factstream",
        "cols" : "factstream.stream_id,factstream.listen_duration"
    },
     {
        "table" : "spotifycatalog.silver.dimuser",
        "alias" : "dimuser",
        "cols" : "dimuser.user_id,dimuser.user_name",
        "condition" :"factstream.user_id = dimuser.user_id"
    },
       {
        "table" : "spotifycatalog.silver.dimtrack",
        "alias" : "dimtrack",
        "cols" : "dimtrack.track_id,dimtrack.track_name",
        "condition" :"factstream.track_id = dimtrack.track_id"
    }
]

In [0]:
query_text= """
              SELECT
               {% for item in parameter %}
  {{ item.cols }}{% if not loop.last %},{% endif %}
{% endfor %}
                FROM 
                    {% for item in parameter %} 
                          {% if loop.first%}
                              {{item['table']}} as {{item['alias']}}
                          {%endif%}
                    {% endfor %}
                    {% for item in parameter %}
                        {%if not loop.first%}       
                        LEFT JOIN 
                               {{item['table']}} as {{item['alias']}}
                        ON {{item['condition']}}
                        {%endif%}
                    {% endfor %}

"""

In [0]:
jinja_sql= Template(query_text)
query=jinja_sql.render(parameter=parameter)
print(query)

In [0]:
display(spark.sql(query))  # Check the query string for syntax errors